In [283]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [562]:
data = pd.read_csv('data/train.csv')
data.head()

,session_id,startTime,endTime,ProductList,gender
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,male
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,female
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,female
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,male


In [563]:
data.shape

(10500, 5)

In [564]:
data['session_id'].nunique()

10500

### There are no repeated session ids and so we can drop this column

In [565]:
data['start_datetime'] = pd.to_datetime(data['startTime'])
data['end_datetime'] = pd.to_datetime(data['endTime'])

In [566]:
data['start_date'] = [d.date() for d in data['start_datetime']]
data['start_time'] = [d.time() for d in data['start_datetime']]
data['end_date'] = [d.date() for d in data['end_datetime']]
data['end_time'] = [d.time() for d in data['end_datetime']]

In [567]:
data.drop(['startTime', 'endTime', 'start_datetime', 'end_datetime'], axis=1, inplace=True)

In [568]:
data['start_Y'] = data['start_date'].apply(lambda x: x.year)
data['start_M'] = data['start_date'].apply(lambda x: x.month)
data['start_D'] = data['start_date'].apply(lambda x: x.day)
data['end_Y'] = data['end_date'].apply(lambda x: x.year)
data['end_M'] = data['end_date'].apply(lambda x: x.month)
data['end_D'] = data['end_date'].apply(lambda x: x.day)
data.drop(['start_date', 'end_date'], axis=1, inplace=True)

In [569]:
data['start_hour'] = data['start_time'].apply(lambda x: x.hour)
data['start_min'] = data['start_time'].apply(lambda x: x.minute)
data['start_sec'] = data['start_time'].apply(lambda x: x.second)
data['end_hour'] = data['end_time'].apply(lambda x: x.hour)
data['end_min'] = data['end_time'].apply(lambda x: x.minute)
data['end_sec'] = data['end_time'].apply(lambda x: x.second)
data.drop(['start_time', 'end_time'], axis=1, inplace=True)

In [570]:
data.head()

,session_id,ProductList,gender,start_Y,start_M,start_D,end_Y,end_M,end_D,start_hour,start_min,start_sec,end_hour,end_min,end_sec
0,u16159,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,2014,12,15,2014,12,15,18,11,0,18,12,0
1,u10253,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,male,2014,12,16,2014,12,16,14,35,0,14,41,0
2,u19037,A00002/B00001/C00020/D16944/,female,2014,1,12,2014,1,12,15,58,0,15,58,0
3,u14556,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,female,2014,11,23,2014,11,23,2,57,0,3,0,0
4,u24295,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,male,2014,12,17,2014,12,17,16,44,0,16,46,0


In [571]:
data.isnull().sum()

session_id     0
ProductList    0
gender         0
start_Y        0
start_M        0
start_D        0
end_Y          0
end_M          0
end_D          0
start_hour     0
start_min      0
start_sec      0
end_hour       0
end_min        0
end_sec        0
dtype: int64

In [572]:
data['ProductList'][0]

'A00002/B00003/C00006/D28435/;A00002/B00003/C00006/D02554/;A00002/B00003/C00006/D28436/;A00002/B00003/C00006/D28437/'

### The products list are separated by semi-colon. We can split each product into a new row

In [573]:
temp = data['ProductList'].str.split(';')
data = data.reindex(data.index.repeat(temp.apply(len)))
data['product_data'] = np.hstack(temp)

In [574]:
data.head()

,session_id,ProductList,gender,start_Y,start_M,start_D,end_Y,end_M,end_D,start_hour,start_min,start_sec,end_hour,end_min,end_sec,product_data
0,u16159,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002/B00003/C00006/D28435/
0,u16159,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002/B00003/C00006/D02554/
0,u16159,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002/B00003/C00006/D28436/
0,u16159,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002/B00003/C00006/D28437/
1,u10253,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,male,2014,12,16,2014,12,16,14,35,0,14,41,0,A00001/B00009/C00031/D29404/


### The product data is separated by forward slash as follows:  category id, sub category id, sub sub category id and product id. We can split this data into 4 columns 

In [575]:
data['category'] = data['product_data'].str.split('/').str[0]
data['sub_category'] = data['product_data'].str.split('/').str[1]
data['sub_sub_category'] = data['product_data'].str.split('/').str[2]
data['product'] = data['product_data'].str.split('/').str[3]

In [576]:
data.drop(['ProductList', 'product_data'], axis=1, inplace=True)

In [577]:
data.head()

,session_id,gender,start_Y,start_M,start_D,end_Y,end_M,end_D,start_hour,start_min,start_sec,end_hour,end_min,end_sec,category,sub_category,sub_sub_category,product
0,u16159,female,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002,B00003,C00006,D28435
0,u16159,female,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002,B00003,C00006,D02554
0,u16159,female,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002,B00003,C00006,D28436
0,u16159,female,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002,B00003,C00006,D28437
1,u10253,male,2014,12,16,2014,12,16,14,35,0,14,41,0,A00001,B00009,C00031,D29404


In [578]:
data['category'].value_counts()

A00002    14931
A00003     3822
A00001     3300
A00005      413
A00004      261
A00006      182
A00011      130
A00010       75
A00007       52
A00009       47
A00008       38
Name: category, dtype: int64

### There are only 11 categories across the entire dataset

In [579]:
data['sub_category'].nunique()

85

In [580]:
data['sub_sub_category'].nunique()

360

In [581]:
data['product'].nunique()

16503

In [582]:
data.head()

,session_id,gender,start_Y,start_M,start_D,end_Y,end_M,end_D,start_hour,start_min,start_sec,end_hour,end_min,end_sec,category,sub_category,sub_sub_category,product
0,u16159,female,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002,B00003,C00006,D28435
0,u16159,female,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002,B00003,C00006,D02554
0,u16159,female,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002,B00003,C00006,D28436
0,u16159,female,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002,B00003,C00006,D28437
1,u10253,male,2014,12,16,2014,12,16,14,35,0,14,41,0,A00001,B00009,C00031,D29404


### Fitting a model without considering start time and end time

In [583]:
train_data = data.drop('gender', axis=1)
train_data.head()

,session_id,start_Y,start_M,start_D,end_Y,end_M,end_D,start_hour,start_min,start_sec,end_hour,end_min,end_sec,category,sub_category,sub_sub_category,product
0,u16159,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002,B00003,C00006,D28435
0,u16159,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002,B00003,C00006,D02554
0,u16159,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002,B00003,C00006,D28436
0,u16159,2014,12,15,2014,12,15,18,11,0,18,12,0,A00002,B00003,C00006,D28437
1,u10253,2014,12,16,2014,12,16,14,35,0,14,41,0,A00001,B00009,C00031,D29404


In [584]:
y = data['gender']

In [585]:
test_data = pd.read_csv('data/test.csv')

test_data['start_datetime'] = pd.to_datetime(test_data['startTime'])
test_data['end_datetime'] = pd.to_datetime(test_data['endTime'])

test_data['start_date'] = [d.date() for d in test_data['start_datetime']]
test_data['start_time'] = [d.time() for d in test_data['start_datetime']]
test_data['end_date'] = [d.date() for d in test_data['end_datetime']]
test_data['end_time'] = [d.time() for d in test_data['end_datetime']]

test_data.drop(['startTime', 'endTime', 'start_datetime', 'end_datetime'], axis=1, inplace=True)

test_data['start_Y'] = test_data['start_date'].apply(lambda x: x.year)
test_data['start_M'] = test_data['start_date'].apply(lambda x: x.month)
test_data['start_D'] = test_data['start_date'].apply(lambda x: x.day)
test_data['end_Y'] = test_data['end_date'].apply(lambda x: x.year)
test_data['end_M'] = test_data['end_date'].apply(lambda x: x.month)
test_data['end_D'] = test_data['end_date'].apply(lambda x: x.day)
test_data.drop(['start_date', 'end_date'], axis=1, inplace=True)

test_data['start_hour'] = test_data['start_time'].apply(lambda x: x.hour)
test_data['start_min'] = test_data['start_time'].apply(lambda x: x.minute)
test_data['start_sec'] = test_data['start_time'].apply(lambda x: x.second)
test_data['end_hour'] = test_data['end_time'].apply(lambda x: x.hour)
test_data['end_min'] = test_data['end_time'].apply(lambda x: x.minute)
test_data['end_sec'] = test_data['end_time'].apply(lambda x: x.second)
test_data.drop(['start_time', 'end_time'], axis=1, inplace=True)


temp = test_data['ProductList'].str.split(';')
test_data = test_data.reindex(test_data.index.repeat(temp.apply(len)))
test_data['product_data'] = np.hstack(temp)

test_data['category'] = test_data['product_data'].str.split('/').str[0]
test_data['sub_category'] = test_data['product_data'].str.split('/').str[1]
test_data['sub_sub_category'] = test_data['product_data'].str.split('/').str[2]
test_data['product'] = test_data['product_data'].str.split('/').str[3]

test_data.drop(['ProductList', 'product_data'], axis=1, inplace=True)

test_data.head()

,session_id,start_Y,start_M,start_D,end_Y,end_M,end_D,start_hour,start_min,start_sec,end_hour,end_min,end_sec,category,sub_category,sub_sub_category,product
0,u12112,2014,8,12,2014,8,12,13,36,0,13,36,0,A00002,B00003,C00006,D19956
1,u19725,2014,12,19,2014,12,19,13,52,0,13,52,0,A00002,B00005,C00067,D02026
2,u11795,2014,1,12,2014,1,12,10,44,0,10,44,0,A00002,B00002,C00004,D12538
3,u22639,2014,8,12,2014,8,12,20,19,0,20,22,0,A00002,B00003,C00079,D22781
3,u22639,2014,8,12,2014,8,12,20,19,0,20,22,0,A00002,B00003,C00079,D22782


### To perform label encoding, we need to append train and test data and fit label encoder on it

In [586]:
combined_data = train_data.append(test_data)

In [587]:
le_cat = LabelEncoder()
le_subcat = LabelEncoder()
le_subsubcat = LabelEncoder()
le_product = LabelEncoder()
le_gender = LabelEncoder()
le_session = LabelEncoder()

In [588]:
combined_data['category'] = le_cat.fit_transform(combined_data['category'])
combined_data['sub_category'] = le_subcat.fit_transform(combined_data['sub_category'])
combined_data['sub_sub_category'] = le_subsubcat.fit_transform(combined_data['sub_sub_category'])
combined_data['product'] = le_product.fit_transform(combined_data['product'])
combined_data['session_id'] = le_session.fit_transform(combined_data['session_id'])
y = le_gender.fit_transform(y)

In [589]:
combined_data.head()

,session_id,start_Y,start_M,start_D,end_Y,end_M,end_D,start_hour,start_min,start_sec,end_hour,end_min,end_sec,category,sub_category,sub_sub_category,product
0,6158,2014,12,15,2014,12,15,18,11,0,18,12,0,1,2,5,17574
0,6158,2014,12,15,2014,12,15,18,11,0,18,12,0,1,2,5,1789
0,6158,2014,12,15,2014,12,15,18,11,0,18,12,0,1,2,5,17575
0,6158,2014,12,15,2014,12,15,18,11,0,18,12,0,1,2,5,17576
1,252,2014,12,16,2014,12,16,14,35,0,14,41,0,0,8,30,18146


In [590]:
train_data['category'] = le_cat.transform(train_data['category'])
train_data['sub_category'] = le_subcat.transform(train_data['sub_category'])
train_data['sub_sub_category'] = le_subsubcat.transform(train_data['sub_sub_category'])
train_data['product'] = le_product.transform(train_data['product'])

In [591]:
train_data['session_id'] = le_session.transform(train_data['session_id'])

In [592]:
test_data['category'] = le_cat.transform(test_data['category'])

In [593]:
test_data['sub_category'] = le_subcat.transform(test_data['sub_category'])

In [594]:
test_data['sub_sub_category'] = le_subsubcat.transform(test_data['sub_sub_category'])

In [595]:
test_data['product'] = le_product.transform(test_data['product'])

In [596]:
test_data['session_id'] = le_session.transform(test_data['session_id'])

In [597]:
test_data.head()

,session_id,start_Y,start_M,start_D,end_Y,end_M,end_D,start_hour,start_min,start_sec,end_hour,end_min,end_sec,category,sub_category,sub_sub_category,product
0,2111,2014,8,12,2014,8,12,13,36,0,13,36,0,1,2,5,12675
1,9724,2014,12,19,2014,12,19,13,52,0,13,52,0,1,4,66,1407
2,1794,2014,1,12,2014,1,12,10,44,0,10,44,0,1,1,3,8129
3,12638,2014,8,12,2014,8,12,20,19,0,20,22,0,1,2,77,14345
3,12638,2014,8,12,2014,8,12,20,19,0,20,22,0,1,2,77,14346


In [598]:
# train_data.drop(['start_hour', 'start_min', 'start_sec'], axis=1, inplace=True)
# train_data.drop(['end_hour', 'end_min', 'end_sec'], axis=1, inplace=True)

# test_data.drop(['start_hour', 'start_min', 'start_sec'], axis=1, inplace=True)
# test_data.drop(['end_hour', 'end_min', 'end_sec'], axis=1, inplace=True)

train_data.drop(['start_Y', 'start_M', 'start_D'], axis=1, inplace=True)
train_data.drop(['end_Y', 'end_M', 'end_D'], axis=1, inplace=True)

test_data.drop(['start_Y', 'start_M', 'start_D'], axis=1, inplace=True)
test_data.drop(['end_Y', 'end_M', 'end_D'], axis=1, inplace=True)

train_data.drop(['category'], axis=1, inplace=True)
test_data.drop(['category'], axis=1, inplace=True)

In [599]:
lr = LogisticRegression()
lr.fit(train_data, y)

C:\Users\Sufyan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [600]:
y_pred = lr.predict(test_data)

In [601]:
test_data['gender'] = le_gender.inverse_transform(y_pred)

In [602]:
test_data['session_id'] = le_session.inverse_transform(test_data['session_id'])

In [603]:
test_data.head()

,session_id,start_hour,start_min,start_sec,end_hour,end_min,end_sec,sub_category,sub_sub_category,product,gender
0,u12112,13,36,0,13,36,0,2,5,12675,female
1,u19725,13,52,0,13,52,0,4,66,1407,female
2,u11795,10,44,0,10,44,0,1,3,8129,female
3,u22639,20,19,0,20,22,0,2,77,14345,female
3,u22639,20,19,0,20,22,0,2,77,14346,female


In [604]:
test_data.shape

(10204, 11)

In [605]:
test_new = pd.read_csv('data/test.csv')
test_new.head()

,session_id,startTime,endTime,ProductList
0,u12112,08/12/14 13:36,08/12/14 13:36,A00002/B00003/C00006/D19956/
1,u19725,19/12/14 13:52,19/12/14 13:52,A00002/B00005/C00067/D02026/
2,u11795,01/12/14 10:44,01/12/14 10:44,A00002/B00002/C00004/D12538/
3,u22639,08/12/14 20:19,08/12/14 20:22,A00002/B00003/C00079/D22781/;A00002/B00003/C00...
4,u18034,15/12/14 19:33,15/12/14 19:33,A00002/B00001/C00010/D23419/


In [606]:
test_data = test_data.drop_duplicates(subset=['session_id'])

In [607]:
to_submit = test_data[['session_id', 'gender']]

In [608]:
to_submit.head()

,session_id,gender
0,u12112,female
1,u19725,female
2,u11795,female
3,u22639,female
4,u18034,female


In [609]:
to_submit.to_csv('data/datecategory_ignored.csv',index = False)

### Removing only time decreased test accuracy while removing only date increased test accuracy. Should we remove date?